<a href="https://colab.research.google.com/github/thebhulawat/tuner/blob/main/tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 25.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 k

In [ ]:

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: ··········
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
huggingface_dataset_name = "neil-code/dialogsum-test"
dataset = load_dataset(huggingface_dataset_name)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset['train'][0]

{'id': 'train_0',
 'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.",
 'summary': "Mr. Smith'

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False)

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
model_name = 'microsoft/phi-2'
device_map = {"": 0}
original_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                      device_map = device_map,
                                                      quantization_config = bnb_config,
                                                      trust_remote_code = True,
                                                      use_auth_token = True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code= True, padding_side = "left", add_eos_token=True, add_bos_token = True, use_fast = False)
tokenizer.pad_token = tokenizer.eos_token
def gen(model, prompt, maxlen=100, sample = True):
  toks = tokenizer(prompt, return_tensors = "pt")
  res = model.generate(**toks.to("cuda"), max_new_tokens = maxlen, do_sample = sample, num_return_sequences = 1, temperature = 0.1, num_beams = 1, top_p=  0.95).to('cpu')
  return tokenizer.batch_decode(res, skip_special_tokens = True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
%%time
from transformers import set_seed
seed = 42
set_seed(seed)
index = 10
dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']
prompt = f"Instruct: Summarize the following conversation. \n {dialogue} \n Output:\n"
res = gen(original_model, prompt, 100, )
output = res[0].split('Output:\n')[1]
#print(res[0])
dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'prompt:\n{prompt}')
print(dash_line)
print(f'Baseline Human Summary:\n{summary}')
print(dash_line)
print(f'Model Generation - Zero Shot:\n{output}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
prompt:
Instruct: Summarize the following conversation. 
 #Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday 
 Output:

---------------------------------------------------------------------------------------------------
Baseline Human Summary:
#Person1# attends Brian's birthday part

In [ ]:
def create_prompt_formats(sample):
  INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
  INSTRUCTION_KEY = "### Instruct: Summarize the below conversation."
  RESPONSE_KEY = "### OUTPUT:"
  END_KEY = "### END"

  blurb = f"\n{INTRO_BLURB}"
  instruction = f"\n{INSTRUCTION_KEY}"
  input_context = f"{sample['dialogue']}" if sample['dialogue'] else None
  response = f"{RESPONSE_KEY} \n {sample['summary']}"
  end = f"{END_KEY}"

  parts = [part for part in [blurb, instruction, input_context, response, end] if part]

  fomatted_prompt = "\n\n".join(parts)
  sample["text"] = fomatted_prompt

  return sample

In [ ]:
from functools import partial

def get_max_length(model):
  conf = model.config
  max_length = None
  for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
    max_length = getattr(model.config, length_setting, None)
    if max_length:
      print(f"Found max length: {max_length}")
      break
    if not max_length:
      max_length = 1024
      print(f"Using default max length: {max_length}")
    return max_length

def preprocess_batch(batch, tokenizer, max_length):
  return tokenizer(
      batch["text"],
      max_length = max_length,
      truncation = True,
  )

def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset):
  print("Preprocessing dataset....")
  dataset = dataset.map(create_prompt_formats)
  _preprocessing_function = partial(preprocess_batch, max_length = max_length, tokenizer = tokenizer)
  dataset = dataset.map(
      _preprocessing_function,
      batched = True,
      remove_columns = ['id', 'topic', 'dialogue', 'summary']
  )
  dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)
  dataset = dataset.shuffle(seed = seed)
  return dataset



In [ ]:
max_length = get_max_length(original_model)
print(max_length)
train_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset['train'])
eval_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset['validation'])
print(len(dataset['train']))
print(len(dataset['validation']))

Using default max length: 1024
1024
Preprocessing dataset....
Preprocessing dataset....
1999
499


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

original_model = prepare_model_for_kbit_training(original_model)

config = LoraConfig(
    r = 32,
    lora_alpha = 32,
    target_modules = [
        'q_proj',
        'k_proj',
        'v_proj',
        'dense',
    ],
    bias = "none",
    lora_dropout = 0.05,
    task_type = "CAUSAL_LM"
)

original_model.gradient_checkpointing_enable()

peft_model = get_peft_model(original_model, config)


In [ ]:
print(peft_model.print_trainable_parameters())

trainable params: 20,971,520 || all params: 2,800,655,360 || trainable%: 0.7488075933770015
None


In [ ]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'
print(output_dir)
import transformers
peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    max_steps = 1000,
    learning_rate = 2e-4,
    optim = "paged_adamw_8bit",
    logging_steps = 25,
    logging_dir = "./logs/",
    save_strategy = "steps",
    save_steps = 25,
    evaluation_strategy = "steps",
    eval_steps = 25,
    do_eval = True,
    gradient_checkpointing = True,
    report_to = "none",
    overwrite_output_dir = True,
    group_by_length = True
)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model = peft_model,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    args = peft_training_args,
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm = False)
)


./peft-dialogue-summary-training-1712658356


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
peft_trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

Step,Training Loss,Validation Loss
25,1.702400,1.387932
50,1.205700,1.399293
75,1.435800,1.342684
100,1.199400,1.351588
125,1.438400,1.335986
150,1.172600,1.343526
175,1.424300,1.330589
200,1.195300,1.333539
225,1.435800,1.327271
250,1.171600,1.328786


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

Step,Training Loss,Validation Loss
25,1.702400,1.387932
50,1.205700,1.399293
75,1.435800,1.342684
100,1.199400,1.351588
125,1.438400,1.335986
150,1.172600,1.343526
175,1.424300,1.330589
200,1.195300,1.333539
225,1.435800,1.327271
250,1.171600,1.328786


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

base_model_id = "microsoft/phi-2"
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map = 'auto',
    quantization_config = bnb_config,
    trust_remote_code = True,
    use_auth_token = True
    )

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token = True, trust_remote_code = True, use_fast = False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token
ft_model = PeftModel.from_pretrained(base_mode, '')
---------------------
%%time
from transformers import set_seed
set_seed(seed)

index = 5
dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"Instruct: Summarize the following conversation.\n {dialogue}\nOutput:\n"

ft_model_res = gen(ft_model, prompt, 100, )
ft_model_output = peft_model_res[0].split('Output:\n')[1]
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'Dialogue:{dialogue}')
print(dash_line)
print(f'Human Summary:{summary}')
print(dash_line)
print(f'Model Summary:{prefix}')

-------------------

original_model = AutoModelForCausalLM.fromP_pretrained(base_model_id,
                                                       device_map = 'auto',
                                                       quantization_config = bnb_config,
                                                       trust_remote_code = True,
                                                       use_auth_token = True)

import pandas as pd

dialogues = dataset['test'][0:10]['dialogue']
human_summaries = dataset['test'][0:10]['summary']

original_model_sumamrries = []
instruct_model_sumamrries = []
peft_model_sumamrries = []

for idx, dialogue in enumerate(dialogues):
  human_summary = human_summarries[idx]
  prompt = f"Instruct: Summarize the following conversation.\n{dialogue}\nOutput:\n"
  original_model_res = gen(original_model, 100, )
  original_model_summary = original_model_res[0].split('Output:\n')[1]

  peft_model_res = gen(ft_model,prompt,100,)
  peft_model_output = peft_model_res[0].split('Output:\n')[1]
  print(peft_model_output)
  peft_model_text_output, success, result = peft_model_output.partition('###')

  original_model_summaries.append(original_model_text_output)
  peft_model_summaries.append(peft_model_text_output)

zipper_summaries = list(zip(human_summaries, original_model_summaries,peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_summaries', 'original_model_summaries', 'peft_model_summaries'])

import evaluate

rogue = evaluate.load('rouge')

original_model_results = rogue.compute(
    predictions=original_model_summaries,
    references=human_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)


peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')



